<a href="https://colab.research.google.com/github/Sidharth1999/Capstone-3/blob/main/Image_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import pathlib
import imageio
import functools
import math
from tensorflow.keras.utils import to_categorical

In [2]:
dataDir = "/content/drive/My Drive/Springboard-Capstone-3/data"
projectDir = "/content/drive/My Drive/Springboard-Capstone-3"

# **Download Dataset from Kaggle**

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sidharthr1999","key":"b06c0eb687ec8653837b4badf109296d"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Verify that kaggle commands work
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              229KB  2021-06-01 11:18:46           6831  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           4086  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1364  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           2128  
mathurinache/twitter-edge-nodes                             Twitter Edge Nod

In [ ]:
dataset1 = "iamsouravbanerjee/indian-food-images-dataset"

In [ ]:
#Download dataset
!kaggle datasets download $dataset1 -p "$dataDir" --unzip

 97% 343M/354M [00:02<00:00, 128MB/s]
100% 354M/354M [00:03<00:00, 123MB/s]


# **Data Preparation**

In [ ]:
#Obtain all 80 categories
categories = []
for file in os.listdir(dataDir): categories.append(file)
print(categories)

['adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi', 'aloo_shimla_mirch', 'aloo_tikki', 'anarsa', 'ariselu', 'bandar_laddu', 'basundi', 'bhatura', 'bhindi_masala', 'biryani', 'boondi', 'butter_chicken', 'chak_hao_kheer', 'cham_cham', 'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala', 'chicken_tikka', 'chicken_tikka_masala', 'chikki', 'daal_baati_churma', 'daal_puri', 'dal_makhani', 'dal_tadka', 'dharwad_pedha', 'doodhpak', 'double_ka_meetha', 'dum_aloo', 'gajar_ka_halwa', 'gavvalu', 'ghevar', 'gulab_jamun', 'imarti', 'jalebi', 'kachori', 'kadai_paneer', 'kadhi_pakoda', 'kajjikaya', 'kakinada_khaja', 'kalakand', 'karela_bharta', 'kofta', 'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha', 'lyangcha', 'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua', 'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan', 'navrattan_korma', 'palak_paneer', 'paneer_butter_masala', 'phirni', 'pithe', 'poha', 'poornalu', 'pootharekulu', 'qubani_ka_meetha', 'rabri', 'ras_malai', 'ras

In [ ]:
#Extract all 4000 image paths for each of the 80 categories
def imagePathsFromCategory(category):
  generator = pathlib.Path(dataDir + '/' + category).glob('*.jpg')
  sorted_paths = sorted([x for x in generator])
  return sorted_paths 

image_paths = [imagePathsFromCategory(category) for category in categories]

#Confirm all 4000 image paths were extracted:
print(functools.reduce(lambda a, b: a+b, [len(categoryPaths) for categoryPaths in image_paths]))

4000


In [ ]:
#Split each of the 80 categories into train, validation, and test
from sklearn.model_selection import train_test_split
train = []
validation = []
test = []
for i in range(len(image_paths)):
  train_images, test_images, _, _ = train_test_split(image_paths[i], range(len(image_paths[i])), test_size=0.20, random_state=42)
  train_images, validation_images, _, _ = train_test_split(train_images, range(len(train_images)), test_size=0.20, random_state=42)
  train.append(train_images)
  validation.append(validation_images)
  test.append(test_images)

In [ ]:
#Make separate directories for each set
os.mkdir(os.path.join(projectDir, "train"))
os.mkdir(os.path.join(projectDir, "validation"))
os.mkdir(os.path.join(projectDir, "test"))

In [5]:
trainDir = projectDir + "/" + "train"
validationDir = projectDir + "/" + "validation"
testDir = projectDir + "/" + "test"

In [ ]:
#Insert data from each category into the newly created train, validation, and test folders
for i in range(len(categories)):
  category = categories[i]
  train_set = train[i]
  validation_set = validation[i]
  test_set = test[i]
  trainDestDir = os.path.join(trainDir, category)
  valDestDir = os.path.join(validationDir, category)
  testDestDir = os.path.join(testDir, category)
  os.mkdir(trainDestDir)
  os.mkdir(valDestDir)
  os.mkdir(testDestDir)

  for j in range(len(train_set)):
    impath = os.path.join(trainDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(train_set[j]))

  for j in range(len(validation_set)):
    impath = os.path.join(valDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(validation_set[j]))

  for j in range(len(test_set)):
    impath = os.path.join(testDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(test_set[j]))

# **Modeling**

**Potential Configurable Parameters:**

1.   Train/Dev/Test split-ratio
2.   Number of layers
3.   Number of nodes per layer
4.   Optimizer
5.   Learning rate of optimizer
6.   Target image size
7.   Generator morphological transformations

**We are going to try 4 different CNN architectures:**


1.   VGG16
2.   DenseNet201
3.   ResNet50
4.   InceptionV3



In [3]:
#Make a directory to save different CNN model features
cnnFeaturesDir = os.path.join(projectDir, 'CNN Architecture Features')

In [ ]:
os.mkdir(cnnFeaturesDir)

## **VGG-16**

In [31]:
input_shape = (80, 80, 3)
learning_rate = 0.001
fine_tune = 1
dropout = 0.2
layer1 = 4096
layer2 = 1072

#Build data generators
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16

train_datagen = ImageDataGenerator(
        preprocessing_function = vgg16.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = vgg16.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


### **Transfer Learning Method #2**

In [ ]:
vggModel = tf.keras.applications.VGG16(input_shape=input_shape,include_top=False,weights='imagenet')
if fine_tune > 0:
  for layer in vggModel.layers[:-fine_tune]: layer.trainable = False
else:
  for layer in vggModel.layers: layer.trainable = False


inputs = tf.keras.Input(shape=input_shape)
x = vggModel(inputs, training=False)
'''x = tf.keras.layers.GlobalAveragePooling2D()(x)'''
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(layer1, activation='relu')(x)
x = tf.keras.layers.Dense(layer2, activation='relu')(x)
x = tf.keras.layers.Dropout(dropout)(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model1 = tf.keras.Model(inputs, outputs)

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss='categorical_crossentropy', metrics=['accuracy'])

history = model1.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model1.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

Epoch 1/50
80/80 [==============================] - 32s 382ms/step - loss: 4.3396 - accuracy: 0.0336 - val_loss: 3.8591 - val_accuracy: 0.0859
Epoch 2/50
80/80 [==============================] - 30s 378ms/step - loss: 3.6255 - accuracy: 0.1086 - val_loss: 3.4321 - val_accuracy: 0.1547
Epoch 3/50
80/80 [==============================] - 30s 379ms/step - loss: 3.0967 - accuracy: 0.1961 - val_loss: 3.2588 - val_accuracy: 0.2047
Epoch 4/50
80/80 [==============================] - 30s 379ms/step - loss: 2.6910 - accuracy: 0.2883 - val_loss: 3.2435 - val_accuracy: 0.1969
Epoch 5/50
80/80 [==============================] - 30s 379ms/step - loss: 2.3230 - accuracy: 0.3633 - val_loss: 3.0601 - val_accuracy: 0.2313
Epoch 6/50
80/80 [==============================] - 30s 378ms/step - loss: 1.9902 - accuracy: 0.4355 - val_loss: 3.0111 - val_accuracy: 0.2406
Epoch 7/50
80/80 [==============================] - 30s 378ms/step - loss: 1.6641 - accuracy: 0.5312 - val_loss: 3.2029 - val_accuracy: 0.2562

### **Prediction Results**

In [18]:
results = model1.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

25/25 [==============================] - 6s 253ms/step - loss: 5.9049 - accuracy: 0.2512
test loss, test acc: [5.9049482345581055, 0.2512499988079071]


In [9]:
model1.save(projectDir + '/' + 'Models/VGG16ModelV1.h5')

## **DenseNet201**

In [23]:
input_shape = (40, 40, 3)
learning_rate = 0.001
fine_tune = 1
dropout = 0.2
layer1 = 4096
layer2 = 1072

#Build data generators
from tensorflow.keras.applications import densenet

train_datagen = ImageDataGenerator(
        preprocessing_function = densenet.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = densenet.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


### **Transfer Learning Method #2**

In [22]:
DN201Model = tf.keras.applications.DenseNet201(input_shape=input_shape,include_top=False,weights='imagenet')

if fine_tune > 0:
  for layer in DN201Model.layers[:-fine_tune]: layer.trainable = False
else:
  for layer in DN201Model.layers: layer.trainable = False

inputs = tf.keras.Input(shape=input_shape)
x = DN201Model(inputs, training=False)
'''x = tf.keras.layers.GlobalAveragePooling2D()(x)'''
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(layer1, activation='relu')(x)
x = tf.keras.layers.Dense(layer2, activation='relu')(x)
x = tf.keras.layers.Dropout(dropout)(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model2 = tf.keras.Model(inputs, outputs)

model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss='categorical_crossentropy', metrics=['accuracy'])

history = model2.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model2.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

74842112/74836368 [==============================] - 0s 0us/step
Epoch 1/50
80/80 [==============================] - 41s 391ms/step - loss: 4.4532 - accuracy: 0.0070 - val_loss: 4.3833 - val_accuracy: 0.0109
Epoch 2/50
80/80 [==============================] - 29s 361ms/step - loss: 4.3865 - accuracy: 0.0113 - val_loss: 4.3811 - val_accuracy: 0.0125
Epoch 3/50
80/80 [==============================] - 29s 361ms/step - loss: 4.3819 - accuracy: 0.0094 - val_loss: 4.3704 - val_accuracy: 0.0203
Epoch 4/50
80/80 [==============================] - 29s 362ms/step - loss: 4.3597 - accuracy: 0.0160 - val_loss: 4.3437 - val_accuracy: 0.0188
Epoch 5/50
80/80 [==============================] - 29s 358ms/step - loss: 4.3361 - accuracy: 0.0145 - val_loss: 4.3168 - val_accuracy: 0.0188
Epoch 6/50
80/80 [==============================] - 29s 363ms/step - loss: 4.2973 - accuracy: 0.0176 - val_loss: 4.3055 - val_accuracy: 0.0281
Epoch 7/50
80/80 [==============================] - 29s 357ms/step - loss: 4.

KeyboardInterrupt: ignored

### **Prediction Results**

In [ ]:
results = model2.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

## **ResNet50**

In [27]:
input_shape = (40, 40, 3)
learning_rate = 0.001
fine_tune = 1
dropout = 0.2
layer1 = 4096
layer2 = 1072

#Build data generators
from tensorflow.keras.applications import resnet

train_datagen = ImageDataGenerator(
        preprocessing_function = resnet.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = resnet.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


### **Transfer Learning Method #2**

In [28]:
RN50Model = tf.keras.applications.ResNet50(input_shape=input_shape,include_top=False,weights='imagenet')

if fine_tune > 0:
  for layer in RN50Model.layers[:-fine_tune]: layer.trainable = False
else:
  for layer in RN50Model.layers: layer.trainable = False

inputs = tf.keras.Input(shape=input_shape)
x = RN50Model(inputs, training=False)
'''x = tf.keras.layers.GlobalAveragePooling2D()(x)'''
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(layer1, activation='relu')(x)
x = tf.keras.layers.Dense(layer2, activation='relu')(x)
x = tf.keras.layers.Dropout(dropout)(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model3 = tf.keras.Model(inputs, outputs)

model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss='categorical_crossentropy', metrics=['accuracy'])

history = model3.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model3.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

94773248/94765736 [==============================] - 1s 0us/step
Epoch 1/20
80/80 [==============================] - 32s 364ms/step - loss: 4.4977 - accuracy: 0.0098 - val_loss: 4.3823 - val_accuracy: 0.0141
Epoch 2/20
80/80 [==============================] - 28s 353ms/step - loss: 4.3865 - accuracy: 0.0113 - val_loss: 4.3854 - val_accuracy: 0.0156
Epoch 3/20
80/80 [==============================] - 28s 354ms/step - loss: 4.3837 - accuracy: 0.0113 - val_loss: 4.3770 - val_accuracy: 0.0109
Epoch 4/20
80/80 [==============================] - 28s 355ms/step - loss: 4.3778 - accuracy: 0.0152 - val_loss: 4.3766 - val_accuracy: 0.0203
Epoch 5/20
80/80 [==============================] - 28s 352ms/step - loss: 4.3611 - accuracy: 0.0152 - val_loss: 4.4004 - val_accuracy: 0.0172
Epoch 6/20
80/80 [==============================] - 28s 353ms/step - loss: 4.3603 - accuracy: 0.0141 - val_loss: 4.3721 - val_accuracy: 0.0156
Epoch 7/20
80/80 [==============================] - 28s 352ms/step - loss: 4.

KeyboardInterrupt: ignored

### **Prediction Results**

In [ ]:
results = model3.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

## **InceptionV3**

In [11]:
input_shape = (75, 75, 3)
learning_rate = 0.001
fine_tune = 1
dropout = 0.2
layer1 = 4096
layer2 = 1072

#Build data generators
from tensorflow.keras.applications import inception_v3

train_datagen = ImageDataGenerator(
        preprocessing_function = inception_v3.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = inception_v3.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=input_shape[:2]),
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


### **Transfer Learning Method #2**

In [13]:
IV3Model = tf.keras.applications.InceptionV3(input_shape=input_shape,include_top=False,weights='imagenet')

if fine_tune > 0:
  for layer in IV3Model.layers[:-fine_tune]: layer.trainable = False
else:
  for layer in IV3Model.layers: layer.trainable = False

inputs = tf.keras.Input(shape=input_shape)
x = IV3Model(inputs, training=False)
'''x = tf.keras.layers.GlobalAveragePooling2D()(x)'''
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(layer1, activation='relu')(x)
x = tf.keras.layers.Dense(layer2, activation='relu')(x)
x = tf.keras.layers.Dropout(dropout)(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model4 = tf.keras.Model(inputs, outputs)

model4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),loss='categorical_crossentropy', metrics=['accuracy'])

history = model4.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model4.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

87916544/87910968 [==============================] - 1s 0us/step
Epoch 1/30
80/80 [==============================] - 66s 771ms/step - loss: 4.6490 - accuracy: 0.0133 - val_loss: 4.5731 - val_accuracy: 0.0141
Epoch 2/30
80/80 [==============================] - 64s 797ms/step - loss: 4.4894 - accuracy: 0.0188 - val_loss: 4.4576 - val_accuracy: 0.0266
Epoch 3/30
80/80 [==============================] - 60s 751ms/step - loss: 4.4007 - accuracy: 0.0270 - val_loss: 4.3458 - val_accuracy: 0.0344
Epoch 4/30
80/80 [==============================] - 61s 767ms/step - loss: 4.3144 - accuracy: 0.0316 - val_loss: 4.3199 - val_accuracy: 0.0453
Epoch 5/30
80/80 [==============================] - 62s 769ms/step - loss: 4.2712 - accuracy: 0.0383 - val_loss: 4.3209 - val_accuracy: 0.0266
Epoch 6/30
80/80 [==============================] - 63s 791ms/step - loss: 4.2157 - accuracy: 0.0492 - val_loss: 4.3222 - val_accuracy: 0.0437
Epoch 7/30
80/80 [==============================] - 62s 776ms/step - loss: 4.

### **Prediction Results**

In [14]:
results = model4.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

25/25 [==============================] - 14s 555ms/step - loss: 4.2989 - accuracy: 0.0800
test loss, test acc: [4.298933029174805, 0.07999999821186066]
